In [1]:
import numpy as np
from typing import Tuple, Dict
import matplotlib.pyplot as plt
import pandas as pd
from keras.callbacks import EarlyStopping, ModelCheckpoint
from einops import rearrange, repeat
from ToApps.to_apps import to_slack
# from genetic_optimizer import Genetic_Algorithm
import tensorflow as tf
from tensorflow.math import logical_not
from tensorflow.config.experimental import get_memory_info
from ktfuncs import kt_gru, kt_bigru, kt_lstm, kt_bilstm
from builderfuncs import restore_model, EarlyStopAndSave
import keras_tuner as kt
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))


df = pd.read_csv('data/scaled_U2_data.csv', index_col=0)
df.drop("UNNAMED: 0", axis=1, inplace=True)

devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(devices[0], True)

print("Num GPUs Available: ", len(devices))

Num GPUs Available:  1


In [2]:
def ttv_split(ds: tf.data.Dataset, ds_size: int, 
              train_split: float = 0.8, 
              val_split: float = 0.1, 
              test_split: float = 0.1) -> Tuple[tf.data.Dataset, 
                                                tf.data.Dataset, 
                                                tf.data.Dataset]:

    assert (train_split + test_split + val_split) == 1
   
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds


def split_window(features: tf.Tensor) -> Tuple[Dict[tf.Tensor, tf.Tensor],
                                               tf.Tensor]:
    inputs = features[:12, 1:]
    state_labels = features[12, 75:]
    targ_labels = tf.expand_dims(features[11, 0], axis=0)
    labels = tf.concat([targ_labels, state_labels], axis=0)
    inputs.set_shape([12, 247])
    labels.set_shape([174])
    

    return inputs, labels

                        
def make_dataset(data: pd.DataFrame, length: int, 
                 batch_size: int = 64, multistep: bool = True) -> Tuple[tf.data.Dataset, 
                                                                        tf.data.Dataset, 
                                                                        tf.data.Dataset]:
    data = np.array(data.iloc[:, :], dtype=np.float32)
    ds = tf.keras.utils.timeseries_dataset_from_array(data=data,
                                                        targets=None,
                                                        sequence_length=length,
                                                        sequence_stride=1,
                                                        shuffle=True,
                                                        seed=1,
                                                        batch_size=None)

    ds = ds.filter(lambda x: tf.reduce_all(logical_not(tf.math.is_nan(x))))
    ds = ds.map(split_window).batch(batch_size)
    # ds = ds.apply(tf.data.experimental.assert_cardinality(170680//batch_size + 1))
    train_ds, val_ds, test_ds = ttv_split(ds, 170680//batch_size, train_split=0.8, val_split=0.1, test_split=0.1)
    train_ds = train_ds
    return train_ds, val_ds, test_ds

In [3]:
import os
import datetime
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
# train_print_callback = tf.keras.callbacks.LambdaCallback(on_train_begin=lambda logs: to_slack(f'{logs}'), 
#                                                          on_train_end=lambda logs: to_slack(f'{logs}'))

filepath = "best.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
train_data, val_data, test_data = make_dataset(df, 13, batch_size=64)

In [4]:
from builderfuncs import save_whole_model
from keras_tuner.engine import tuner_utils
import GPUtil
gpu = GPUtil.getGPUs()[0]
from time import sleep
from ToApps.to_apps import to_slack

In [4]:


def _build_and_fit_model(self, trial, *args, **kwargs):
    """For AutoKeras to override.
    DO NOT REMOVE this function. AutoKeras overrides the function to tune
    tf.data preprocessing pipelines, preprocess the dataset to obtain
    the input shape before building the model, adapt preprocessing layers,
    and tune other fit_args and fit_kwargs.
    Args:
        trial: A `Trial` instance that contains the information needed to
            run this trial. `Hyperparameters` can be accessed via
            `trial.hyperparameters`.
        *args: Positional arguments passed by `search`.
        **kwargs: Keyword arguments passed by `search`.
    Returns:
        The fit history.
    """
    
    
    string = f"GRU_trial_{trial.trial_id}\n"
    string += "------------------------------------------------\n"
    if trial.hyperparameters.values:
            for hp, value in trial.hyperparameters.values.items():
                string += f"{hp}: {value}\n"
    
    hp = trial.hyperparameters
    model = self._try_build(hp)
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
    tuner_utils.validate_trial_results(
        results, self.oracle.objective, "HyperModel.fit()"
    )
    try:
        path = os.path.join("grudir", f"trial_{trial.trial_id}")
        os.mkdir(path) 
    except FileExistsError:
         pass
    save_whole_model(model, f"grudir/trial_{trial.trial_id}")

    string += "------------------------------------------------\n"
    result_dict = tuner_utils.convert_to_metrics_dict(results, self.oracle.objective)
    string += f"loss: {result_dict['loss']:.6f}\n"
    string += f"val_loss: {result_dict['val_loss']:.6f}\n"
             
    string += "------------------------------------------------\n"

    while GPUtil.getGPUs()[0].temperature > 89.0:
        to_slack(str(GPUtil.getGPUs()[0].temperature))
        sleep(10)
    try:
        to_slack(string)
    except:
         print("cannot connect to slack")
    
    return results

In [5]:
import types
import traceback
tuner = kt.BayesianOptimization(kt_gru,
                                objective=kt.Objective('val_loss', 'min'),
                                max_trials=20,
                                num_initial_points=8,
                                directory='directory',
                                project_name="gru_trial_1",
                                seed=1)
tuner._build_and_fit_model = types.MethodType(_build_and_fit_model, tuner)

try:
    tuner.search(train_data,
             epochs=15,
             verbose=2,
             validation_data=val_data,
             callbacks=[EarlyStopAndSave(filepath="model_folder", patience=3, lim=0.005, minormax="max")])
except:
    # printing stack trace
    for i in range(5):
        to_slack("GRU ERROR")
    traceback.print_exc()

Trial 20 Complete [00h 46m 56s]
val_loss: 0.001161165302619338

Best val_loss So Far: 0.0011481570545583963
Total elapsed time: 13h 47m 03s
INFO:tensorflow:Oracle triggered exit


In [ ]:
tuner.get_best_models

In [5]:
def _build_and_fit_model(self, trial, *args, **kwargs):
    """For AutoKeras to override.
    DO NOT REMOVE this function. AutoKeras overrides the function to tune
    tf.data preprocessing pipelines, preprocess the dataset to obtain
    the input shape before building the model, adapt preprocessing layers,
    and tune other fit_args and fit_kwargs.
    Args:
        trial: A `Trial` instance that contains the information needed to
            run this trial. `Hyperparameters` can be accessed via
            `trial.hyperparameters`.
        *args: Positional arguments passed by `search`.
        **kwargs: Keyword arguments passed by `search`.
    Returns:
        The fit history.
    """
    
    
    string = f"BDGRU_trial_{trial.trial_id}\n"
    string += "------------------------------------------------\n"
    if trial.hyperparameters.values:
            for hp, value in trial.hyperparameters.values.items():
                string += f"{hp}: {value}\n"
    
    hp = trial.hyperparameters
    model = self._try_build(hp)
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
    tuner_utils.validate_trial_results(
        results, self.oracle.objective, "HyperModel.fit()"
    )
    try:
        path = os.path.join("bigrudir", f"trial_{trial.trial_id}")
        os.mkdir(path) 
    except FileExistsError:
         pass
    save_whole_model(model, f"bigrudir/trial_{trial.trial_id}")

    string += "------------------------------------------------\n"
    result_dict = tuner_utils.convert_to_metrics_dict(results, self.oracle.objective)
    string += f"loss: {result_dict['loss']:.6f}\n"
    string += f"val_loss: {result_dict['val_loss']:.6f}\n"
             
    string += "------------------------------------------------\n"

    while GPUtil.getGPUs()[0].temperature > 89.0:
        to_slack(str(GPUtil.getGPUs()[0].temperature))
        sleep(10)
    try:
        to_slack(string)
    except:
         print("cannot connect to slack")
    
    return results

In [6]:
import types
tuner = kt.BayesianOptimization(kt_bigru,
                                objective=kt.Objective('val_loss', 'min'),
                                max_trials=20,
                                num_initial_points=8,
                                directory='directory',
                                project_name="bigru_trial_1",
                                seed=1)
tuner._build_and_fit_model = types.MethodType(_build_and_fit_model, tuner)

try:
    tuner.search(train_data,
             epochs=10,
             verbose=2,
             validation_data=val_data,
             callbacks=[EarlyStopAndSave(filepath="model_folder", patience=2, lim=0.005, minormax="max")])
except:
    # printing stack trace
    for i in range(5):
        to_slack("BIGRU ERROR")
    traceback.print_exc()

Trial 20 Complete [00h 51m 06s]
val_loss: 0.0010514005552977324

Best val_loss So Far: 0.0010392921976745129
Total elapsed time: 13h 36m 08s
INFO:tensorflow:Oracle triggered exit


In [5]:
def _build_and_fit_model(self, trial, *args, **kwargs):
    """For AutoKeras to override.
    DO NOT REMOVE this function. AutoKeras overrides the function to tune
    tf.data preprocessing pipelines, preprocess the dataset to obtain
    the input shape before building the model, adapt preprocessing layers,
    and tune other fit_args and fit_kwargs.
    Args:
        trial: A `Trial` instance that contains the information needed to
            run this trial. `Hyperparameters` can be accessed via
            `trial.hyperparameters`.
        *args: Positional arguments passed by `search`.
        **kwargs: Keyword arguments passed by `search`.
    Returns:
        The fit history.
    """
    
    
    string = f"LSTM_trial_{trial.trial_id}\n"
    string += "------------------------------------------------\n"
    if trial.hyperparameters.values:
            for hp, value in trial.hyperparameters.values.items():
                string += f"{hp}: {value}\n"
    
    hp = trial.hyperparameters
    model = self._try_build(hp)
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
    tuner_utils.validate_trial_results(
        results, self.oracle.objective, "HyperModel.fit()"
    )
    try:
        path = os.path.join("lstmdir", f"trial_{trial.trial_id}")
        os.mkdir(path) 
    except FileExistsError:
         pass
    save_whole_model(model, f"lstmdir/trial_{trial.trial_id}")

    string += "------------------------------------------------\n"
    result_dict = tuner_utils.convert_to_metrics_dict(results, self.oracle.objective)
    string += f"loss: {result_dict['loss']:.6f}\n"
    string += f"val_loss: {result_dict['val_loss']:.6f}\n"
             
    string += "------------------------------------------------\n"

    while GPUtil.getGPUs()[0].temperature > 89.0:
        to_slack(str(GPUtil.getGPUs()[0].temperature))
        sleep(10)
    try:
        to_slack(string)
    except:
         print("cannot connect to slack")
    
    return results

In [6]:
import types
import traceback
tuner = kt.BayesianOptimization(kt_lstm,
                                objective=kt.Objective('val_loss', 'min'),
                                max_trials=20,
                                num_initial_points=8,
                                directory='directory',
                                project_name="lstm_trial_1",
                                seed=1)
tuner._build_and_fit_model = types.MethodType(_build_and_fit_model, tuner)

try:
    tuner.search(train_data,
             epochs=15,
             verbose=2,
             validation_data=val_data,
             callbacks=[EarlyStopAndSave(filepath="model_folder", patience=3, lim=0.005, minormax="max")])
except:
    # printing stack trace
    for i in range(5):
        to_slack("LSTM ERROR")
    traceback.print_exc()

Trial 20 Complete [00h 53m 27s]
val_loss: 0.0010649551404640079

Best val_loss So Far: 0.0010649551404640079
Total elapsed time: 17h 08m 48s
INFO:tensorflow:Oracle triggered exit


In [7]:
def _build_and_fit_model(self, trial, *args, **kwargs):
    """For AutoKeras to override.
    DO NOT REMOVE this function. AutoKeras overrides the function to tune
    tf.data preprocessing pipelines, preprocess the dataset to obtain
    the input shape before building the model, adapt preprocessing layers,
    and tune other fit_args and fit_kwargs.
    Args:
        trial: A `Trial` instance that contains the information needed to
            run this trial. `Hyperparameters` can be accessed via
            `trial.hyperparameters`.
        *args: Positional arguments passed by `search`.
        **kwargs: Keyword arguments passed by `search`.
    Returns:
        The fit history.
    """
    
    
    string = f"BDLSTM_trial_{trial.trial_id}\n"
    string += "------------------------------------------------\n"
    if trial.hyperparameters.values:
            for hp, value in trial.hyperparameters.values.items():
                string += f"{hp}: {value}\n"
    
    hp = trial.hyperparameters
    model = self._try_build(hp)
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
    tuner_utils.validate_trial_results(
        results, self.oracle.objective, "HyperModel.fit()"
    )
    try:
        path = os.path.join("bilstmdir", f"trial_{trial.trial_id}")
        os.mkdir(path) 
    except FileExistsError:
         pass
    save_whole_model(model, f"bilstmdir/trial_{trial.trial_id}")

    string += "------------------------------------------------\n"
    result_dict = tuner_utils.convert_to_metrics_dict(results, self.oracle.objective)
    string += f"loss: {result_dict['loss']:.6f}\n"
    string += f"val_loss: {result_dict['val_loss']:.6f}\n"
             
    string += "------------------------------------------------\n"

    while GPUtil.getGPUs()[0].temperature > 89.0:
        to_slack(str(GPUtil.getGPUs()[0].temperature))
        sleep(10)
    try:
        to_slack(string)
    except:
         print("cannot connect to slack")
    
    return results

In [8]:
tuner = kt.BayesianOptimization(kt_bilstm,
                                objective=kt.Objective('val_loss', 'min'),
                                max_trials=25,
                                num_initial_points=8,
                                directory='directory',
                                project_name="bilstm_trial_1",
                                seed=1)
tuner._build_and_fit_model = types.MethodType(_build_and_fit_model, tuner)

try:
    tuner.search(train_data,
             epochs=15,
             verbose=2,
             validation_data=val_data,
             callbacks=[EarlyStopAndSave(filepath="model_folder", patience=3, lim=0.005, minormax="max")])
except:
    # printing stack trace
    for i in range(5):
        to_slack("BILSTM ERROR")
    traceback.print_exc()

Trial 25 Complete [00h 58m 14s]
val_loss: 0.0009002964943647385

Best val_loss So Far: 0.0008962851134128869
Total elapsed time: 1d 08h 15m 53s
INFO:tensorflow:Oracle triggered exit


In [4]:
import json
gru_hyperparams = []
bigru_hyperparams = []
lstm_hyperparams = []
bilstm_hyperparams = []
for i in range(25):
    try:
        with open(f"directory/gru_trial_1/trial_{i:02}/trial.json") as f:
            gru_parameters = json.load(f)
        gru_hyperparams.append(gru_parameters)
    except FileNotFoundError:
        print(f"gru_trial_1/trial_{i:02} not found")
    try:
        with open(f"directory/bigru_trial_1/trial_{i:02}/trial.json") as f:
            bigru_parameters = json.load(f)
        bigru_hyperparams.append(bigru_parameters)
    except FileNotFoundError:
        print(f"bigru_trial_1/trial_{i:02} not found")
    try:
        with open(f"directory/lstm_trial_1/trial_{i:02}/trial.json") as f:
            lstm_parameters = json.load(f)
        lstm_hyperparams.append(lstm_parameters)
    except FileNotFoundError:
        print(f"lstm_trial_1/trial_{i:02} not found")
    try:
        with open(f"directory/bilstm_trial_1/trial_{i:02}/trial.json") as f:
            bilstm_parameters = json.load(f)
        bilstm_hyperparams.append(bilstm_parameters)
    except FileNotFoundError:
        print(f"bilstm_trial_1/trial_{i:02} not found")

print(gru_hyperparams)

gru_trial_1/trial_20 not found
bigru_trial_1/trial_20 not found
lstm_trial_1/trial_20 not found
gru_trial_1/trial_21 not found
bigru_trial_1/trial_21 not found
lstm_trial_1/trial_21 not found
gru_trial_1/trial_22 not found
bigru_trial_1/trial_22 not found
lstm_trial_1/trial_22 not found
gru_trial_1/trial_23 not found
bigru_trial_1/trial_23 not found
lstm_trial_1/trial_23 not found
gru_trial_1/trial_24 not found
bigru_trial_1/trial_24 not found
lstm_trial_1/trial_24 not found
[{'trial_id': '00', 'hyperparameters': {'space': [{'class_name': 'Fixed', 'config': {'name': 'lookback', 'conditions': [], 'value': 12}}, {'class_name': 'Fixed', 'config': {'name': 'n_features', 'conditions': [], 'value': 247}}, {'class_name': 'Fixed', 'config': {'name': 'n_manips', 'conditions': [], 'value': 74}}, {'class_name': 'Fixed', 'config': {'name': 'n_targs', 'conditions': [], 'value': 174}}, {'class_name': 'Fixed', 'config': {'name': 'horizon', 'conditions': [], 'value': 12}}, {'class_name': 'Int', 'confi

In [5]:
gru_hyperparams = sorted(gru_hyperparams, key=lambda x: x["score"])
bigru_hyperparams = sorted(bigru_hyperparams, key=lambda x: x["score"])
lstm_hyperparams = sorted(lstm_hyperparams, key=lambda x: x["score"])
bilstm_hyperparams = sorted(bilstm_hyperparams, key=lambda x: x["score"])

In [6]:
table = []
from tabulate import tabulate
for i in range(5):
    line = [gru_hyperparams[i]['trial_id']]
    for key in list(gru_hyperparams[i]['hyperparameters']['values'].keys()):
        line.append(gru_hyperparams[i]['hyperparameters']['values'][key])
    line.append(gru_hyperparams[i]['score'])
    table.append(line)
headers = list(gru_hyperparams[i]['hyperparameters']['values'].keys())
headers.append('score')
print('GRU')
print(tabulate(table, headers=headers))

table = []
for i in range(5):
    line = [bigru_hyperparams[i]['trial_id']]
    for key in list(bigru_hyperparams[i]['hyperparameters']['values'].keys()):
        line.append(bigru_hyperparams[i]['hyperparameters']['values'][key])
    line.append(bigru_hyperparams[i]['score'])
    table.append(line)
headers = list(bigru_hyperparams[i]['hyperparameters']['values'].keys())
headers.append('score')
print('BDGRU')
print(tabulate(table, headers=headers))

table = []
for i in range(5):
    line = [lstm_hyperparams[i]['trial_id']]
    for key in list(lstm_hyperparams[i]['hyperparameters']['values'].keys()):
        line.append(lstm_hyperparams[i]['hyperparameters']['values'][key])
    line.append(lstm_hyperparams[i]['score'])
    table.append(line)
headers = list(lstm_hyperparams[i]['hyperparameters']['values'].keys())
headers.append('score')
print('LSTM')
print(tabulate(table, headers=headers))

table = []
for i in range(5):
    line = [bilstm_hyperparams[i]['trial_id']]
    for key in list(bilstm_hyperparams[i]['hyperparameters']['values'].keys()):
        line.append(bilstm_hyperparams[i]['hyperparameters']['values'][key])
    line.append(bilstm_hyperparams[i]['score'])
    table.append(line)
headers = list(bilstm_hyperparams[i]['hyperparameters']['values'].keys())
headers.append('score')
print('BDLSTM')
print(tabulate(table, headers=headers))

GRU
      lookback    n_features    n_manips    n_targs    horizon    units    recurrent_dropout    dropout  activation    recurrent_activation      learning_rate    beta_1    beta_2    epsilon       score
--  ----------  ------------  ----------  ---------  ---------  -------  -------------------  ---------  ------------  ----------------------  ---------------  --------  --------  ---------  ----------
18          12           247          74        174         12      492                    0        0.1  tanh          sigmoid                          0.0001      0.85     0.995      1e-07  0.00114816
19          12           247          74        174         12      556                    0        0.1  tanh          sigmoid                          0.0001      0.85     0.995      1e-07  0.00116117
10          12           247          74        174         12      428                    0        0.1  tanh          sigmoid                          0.0001      0.85     0.995      1e-0

In [7]:
gru = restore_model(f"grudir/trial_{gru_hyperparams[0]['trial_id']}", "gru")
bigru = restore_model(f"bigrudir/trial_{bigru_hyperparams[0]['trial_id']}", "bigru")
lstm = restore_model(f"lstmdir/trial_{lstm_hyperparams[0]['trial_id']}", "lstm")
bilstm = restore_model(f"bilstmdir/trial_{bilstm_hyperparams[0]['trial_id']}", "bilstm")

Model: "gru_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_input (InputLayer)      [(None, 12, 247)]         0         
                                                                 
 gru_0 (GRU)                 (None, 12, 492)           1093716   
                                                                 
 gru_1 (GRU)                 (None, 492)               1455336   
                                                                 
 output (Dense)              (None, 174)               85782     
                                                                 
Total params: 2,634,834
Trainable params: 2,634,834
Non-trainable params: 0
_________________________________________________________________
None
Model: "bigru_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bigru_input (InputLa

In [8]:
print(gru.evaluate(test_data))
print(bigru.evaluate(test_data))
print(lstm.evaluate(test_data))
print(bilstm.evaluate(test_data))

408/408 [==============================] - 23s 16ms/step - loss: 0.0011
0.001146542839705944
408/408 [==============================] - 27s 33ms/step - loss: 0.0010
0.0010340315056964755
408/408 [==============================] - 25s 27ms/step - loss: 0.0011
0.001065485761500895
408/408 [==============================] - 35s 50ms/step - loss: 9.0482e-04
0.0009048190549947321


In [9]:
import matplotlib.pyplot as plt
test = test_data.unbatch().batch(1)
gru_preds = gru.predict(test[0])
bigru_preds = bigru.predict(test[0])
lstm_preds = lstm.predict(test[0])
bilstm_preds = bilstm.predict(test[0])

26073/26073 [==============================] - 183s 6ms/step
  11674/Unknown - 241s 19ms/step

KeyboardInterrupt: 

In [ ]:
iterator = test.as_numpy_iterator()
elements = [element for element in iterator]

fig, ax = plt.subplots((20, 4), figsize=(10, 20))
for i in range(20):
    ax[i, 0].plot(elements[i][1][0, :, 0])
    ax[i, 0].plot(gru_preds[i])
    ax[i, 1].plot(elements[i][1][0, :, 0])
    ax[i, 1].plot(bigru_preds[i])
    ax[i, 2].plot(elements[i][1][0, :, 0])
    ax[i, 2].plot(lstm_preds[i])
    ax[i, 3].plot(elements[i][1][0, :, 0])
    ax[i, 3].plot(bilstm_preds[i])